In [1]:
#default_exp snowflake.query

In [2]:
#hide
from nbdev.showdoc import *

# ``SnowflakeConnect:``

In [3]:
#export
# from sdsde.imports import *
from functools import partial
from fastcore.basics import store_attr
from fastcore.meta import delegates
from snowflake.sqlalchemy import URL
from sqlalchemy import create_engine
from snowflake import connector
from snowflake.connector.pandas_tools import write_pandas, pd_writer

import concurrent.futures
import pandas as pd
import numpy as np
import gc
import logging
import os

logging.basicConfig(level=logging.INFO)
logging.getLogger("azure.core").setLevel(logging.WARNING)
logging.getLogger("urllib3.connectionpool").setLevel(logging.CRITICAL)
logging.getLogger("snowflake.connector").setLevel(logging.WARNING)
logger = logging.getLogger(__name__)

In [4]:
#export


class SnowflakeConnect:
    """
    Class that holds basic snowflake functionality including testing connection
    and running queries.
    """

    def __init__(self, sfAccount: str, sfUser: str, sfPswd: str,
                 sfWarehouse: str, sfDatabase: str, sfSchema: str,
                 sfRole: str, delimiter: str = ';', logger=None
                 ):
        """
        Creates a connection to snowflake account for a simple class
        for a python users to take advantage of the computation power of
        snowflake. This class is used in all snowflake .py files through
        super().

        Args:
        * sfAccount (str): snowflake credential passed as string. Defaults to None.
        * sfUser (str): snowflake credential passed as string. Defaults to None.
        * sfPswd (str): snowflake credential passed as string. Defaults to None.
        * sfWarehouse (str): snowflake credential passed as string. Defaults to None.
        * sfDatabase (str): snowflake credential passed as string. Defaults to None.
        * sfSchema (str): snowflake credential passed as string. Defaults to None.
        * sfRole (str): snowflake credential passed as string. Defaults to None.
        * delimiter (str, optional): parse delimiter for the query files. Defaults to ';'.
        * logger (logger, optional): pass custom logger as many libraries are set to Warning. Defaults to None.
        """
        store_attr()
        self._logger = logger if logger is not None else logging.getLogger(__name__)
        self.delimiter = delimiter
        try:
            self.test_connection()
        except Exception as e:
            self._logger.error(f'{e}')
            raise

    @staticmethod
    def parse_sql_file(path: str, delimiter: str):
        """
        Helper function to parse string on default delimiter ;
        Args:
        * path (str): path to file that will be parsed
        * delimiter (str): string type to parse end of sql command

        Returns:
        * str: sting of the command
        """

        with open(path, 'r') as file:
            lines = file.readlines()

        single_command = ''
        sql_commands = []

        for i in lines:
            single_command = single_command + i
            if f'{delimiter}' in i:
                sql_commands.append(single_command)
                single_command = ''

        return sql_commands

    @staticmethod
    def single_cmd(sf_connection, sql_command):
        """
        utility to ``execute_concurrent_query`` that handles
        the request to snowflake via the class itself

        Args:
        * sf_connection (SnowFlake Engine): Calls internal class function
        * sql_command (str): query to snowflake

        Returns:
        * Depends on response: pd.DataFrame or a String
        """
        logging.info('new thread started')
        response = sf_connection.run_str_query(sql_command)
        logging.info(f"Finishing {sql_command}")
        return response

    def create_sf_engine(self, role=None, schema=None, password=None,
                         user=None, warehouse=None, database=None):
        "Moving Engine from init to a function to see if this will help"
        engine = create_engine(URL(user=user if user else self.sfUser,
                                   password=password if password else self.sfPswd,
                                   account=self.sfAccount,
                                   warehouse=warehouse if warehouse else self.sfWarehouse,
                                   database=database if database else self.sfDatabase,
                                   schema=schema if schema else self.sfSchema,
                                   role=role if role else self.sfRole))
        self._logger.info("sqlalchemy snowflake engine created")
        return engine

    @delegates(create_sf_engine)
    def test_connection(self, **kwargs):
        "Tests a connection for snowflake from instantiated object"
        engine = partial(self.create_sf_engine,
                         role=kwargs.get('role') if kwargs.get('role') else self.sfRole,
                         schema=kwargs.get('schema') if kwargs.get('schema') else self.sfSchema,
                         password=kwargs.get('password') if kwargs.get('password') else self.sfPswd,
                         user=kwargs.get('user') if kwargs.get('user') else self.sfUser,
                         warehouse=kwargs.get('warehouse') if kwargs.get('warehouse') else self.sfWarehouse,
                         database=kwargs.get('database') if kwargs.get('database') else self.sfDatabase
                         )()
        with engine.connect() as connection:
            try:
                if connection:
                    self._logger.info("connection to snowflake successful")
                else:
                    self._logger.info("connection to snowflake failed")
            finally:
                engine.dispose()
                connection.close()
        pass

    @delegates(create_sf_engine)
    def run_str_query(self, query: str, **kwargs):
        """
        This function will connect to snowflake and run a query that is passed
        in as a string.
        Args:
        * query (str): SQL Query String
        Returns:
        * pd.DataFrame/ None: dependent on the query string
        """
        self._logger.info('testing connection')
        engine = partial(self.create_sf_engine,
                         role=kwargs.get('role') if kwargs.get('role') else self.sfRole,
                         schema=kwargs.get('schema') if kwargs.get('schema') else self.sfSchema,
                         password=kwargs.get('password') if kwargs.get('password') else self.sfPswd,
                         user=kwargs.get('user') if kwargs.get('user') else self.sfUser,
                         warehouse=kwargs.get('warehouse') if kwargs.get('warehouse') else self.sfWarehouse,
                         database=kwargs.get('database') if kwargs.get('database') else self.sfDatabase
                         )()
        with engine.connect() as connection:
            try:
                if connection:
                    self._logger.info('executing query')
                    query = query if str(query).endswith(';') else query + ';'
                    df = pd.read_sql_query(query, engine)
                    self._logger.info("data loaded from snowflake")
                else:
                    self._logger.info("connection to snowflake failed")
            finally:
                engine.dispose()
                connection.close()
                self._logger.info("connection to snowflake has been turned off")
        if df.columns[0] == 'status':
            self._logger.info(df.status.values[0])
        else:
            return df

    @delegates(create_sf_engine)
    def execute_file(self, query_path: str, sql: str = None, **kwargs):
        """
        will run sql file or txt file that is seperated by self.delimenter
        with the option to add a custom sql command if you run an sql command
        then the sql command will be the returned response

        Args:
        * query_path (str): location to sql/txt file to execute. Defaults to None.
        * sql (str, optional): Optional Command to add on top of the file being executed. Defaults to None.

        Returns:
        * pd.DataFrame: returns query results
        """

        self._logger.info("query being parsed")
        engine = partial(self.create_sf_engine,
                         role=kwargs.get('role') if kwargs.get('role') else self.sfRole,
                         schema=kwargs.get('schema') if kwargs.get('schema') else self.sfSchema,
                         password=kwargs.get('password') if kwargs.get('password') else self.sfPswd,
                         user=kwargs.get('user') if kwargs.get('user') else self.sfUser,
                         warehouse=kwargs.get('warehouse') if kwargs.get('warehouse') else self.sfWarehouse,
                         database=kwargs.get('database') if kwargs.get('database') else self.sfDatabase
                         )()
        sql_commands = self.parse_sql_file(query_path, self.delimiter)
        commands = len(sql_commands)
        i = 0
        with engine.connect() as connection:
            try:
                if connection:
                    for z in sql_commands:
                        if i < commands:
                            i += 1
                            df = pd.read_sql_query(z, engine)
                    self._logger.info("sql or txt file excecuted")
                    if sql:
                        self._logger.info(f"Last query response {df}")
                        df = pd.read_sql_query(sql, engine)
                        self._logger.info("data is loaded from snowflake")
                else:
                    self._logger.info("connection to snowflake failed")
                    df = None
                return df

            finally:
                engine.dispose()
                connection.close()
                self._logger.info("connection to snowflake has been turned off")

    @delegates(create_sf_engine)
    def execute_large_query_dump_to_file(self, query_path: str, file_name: str,
                                         chunk_size: int = 150000, **kwargs):
        """
        will run sql file or txt file that is seperated by self.delimenter
        with the option to add a custom sql command if you run an sql command
        then the sql command will be the returned response

        Args:
        * query_path (str): location to sql/txt file to execute. Defaults to None.
        * sql (str, optional): Optional Command to add on top of the file being executed. Defaults to None.

        Returns:
        * pd.DataFrame: returns query results
        """

        self._logger.info("query being parsed")
        engine = partial(self.create_sf_engine,
                         role=kwargs.get('role') if kwargs.get('role') else self.sfRole,
                         schema=kwargs.get('schema') if kwargs.get('schema') else self.sfSchema,
                         password=kwargs.get('password') if kwargs.get('password') else self.sfPswd,
                         user=kwargs.get('user') if kwargs.get('user') else self.sfUser,
                         warehouse=kwargs.get('warehouse') if kwargs.get('warehouse') else self.sfWarehouse,
                         database=kwargs.get('database') if kwargs.get('database') else self.sfDatabase
                         )()
        sql_commands = self.parse_sql_file(query_path, self.delimiter)
        commands = len(sql_commands)
        i = 0
        with engine.connect() as connection:
            try:
                if connection:
                    for z in sql_commands:
                        if i < commands:
                            i += 1
                        chunk_sizer = 0
                        for chunk_df in pd.read_sql_query(z, engine, chunksize=chunk_size):
                            if not os.path.exists(file_name):
                                # TODO: Add Parquet Logic here when needed pandas doesn't support
                                # Pyarrow does sort of and parquet files aren't meant for appending
                                self._logger.info(f'{file_name} being created')
                                chunk_df.to_csv(file_name, index=False)
                            else:
                                self._logger.info(f'{file_name} appending to')
                                chunk_df.to_csv(file_name, header=False, mode='a', index=False)
                            chunk_sizer += chunk_size
                            self._logger.info(f"streamed data total {chunk_sizer}")
                    self._logger.info(f"sql or txt file excecuted and pushed data to {file_name}")
                else:
                    self._logger.info("connection to snowflake failed")

            finally:
                engine.dispose()
                connection.close()
                self._logger.info("connection to snowflake has been turned off")

    def infer_to_snowflake(self,
                           df: pd.DataFrame,
                           table_name: str,
                           if_exists: str = 'append',
                           dtype: dict = None,
                           chunk_size: int = None):
        """
        Args:
        * df (pd.DataFrame): The dataframe that is going to snowflake
        * table_name (str): Table name that will be created in snowflake
        * if_exists (str, optional): How to behave if the table already exists.
                                    * fail: Raise a ValueError.
                                    * replace: Drop the table before inserting new values.
                                    * append: Insert new values to the existing table.
                                    Defaults to 'fail'.
        * dtype (dict, optional): Specifying the datatype for columns. If a dictionary is used, the
                                keys should be the column names and the values should be the
                                SQLAlchemy types or strings for the sqlite3 legacy mode. If a
                                scalar is provided, it will be applied to all columns.
                                Defaults to None.
        * chunk_size (int, optional): Specify the number of rows in each batch to be written
                                    at a time. By default, all rows will be written at once.
                                    Defaults to None.
        """
        df.columns = [x.upper() for x in df.columns]
        self._logger.info(f'Begining upload to {table_name}')
        df.to_sql(table_name.upper(), self.create_sf_engine().connect(), index=False, chunksize=chunk_size,
                  method=pd_writer, if_exists=if_exists)
        self._logger.info(f'Dataframe uploaded to {table_name}')

    @delegates(infer_to_snowflake)
    def large_table_infer_sf(self, df: pd.DataFrame, table_name: str, split: int, **kwargs):
        """
        This take infer_to_snowflake and give the ability to split the loads to snowflake
        this isn't tested for efficiency, but might be useful when not wanting to wait
        for a 100M+ memory object to make it to snowflake. A user can split this into
        as many slices as they choose and it will fail if there is something wrong with
        the upload.

        Args:
        * df (pd.DataFrame): in memory pandas dataframe
        * table_name (str): table name for table being created
        * split (int): how many splits to split the data frame up in
        """
        # Don't know a good cut off point 3.6GB takes ~14 mins at split 7
        if kwargs.get('if_exists') != 'append' and kwargs.get('if_exists') is not None:
            self._logger.warning('only append is supported with large dataframes')
            self._logger.info('if user is trying to replace or fail use infer_to_snowflake')
            return
        if df.shape[0]/split < 50000:
            self._logger.warning('best to use infer_to_snowflake when splits are less than 50k')
        if kwargs.get('if_exists') is None:
            self._logger.info(f'if_exisits defaults to append to {table_name}')
        if_exists = 'append'
        for chunk in np.array_split(df, split):
            gc.collect()
            self.infer_to_snowflake(chunk,
                                    table_name=table_name.upper(),
                                    if_exists=if_exists)

    @delegates(infer_to_snowflake)
    def chunk_csv_infer_sf(self, file_path: str, table_name: str, chunk_size: int, **kwargs):
        """
        Again takes from infer_to_snowflake, but give the user the ability to chunk
        out a file from local machine. Currently not supporting parquet files, but
        it also isn't tested and might be a simple switch to allow parquet reads.

        Args:
        * file_path (str): location of file to upload to snowflake
        * table_name (str): table name for table being created
        * chunk_size (int): number of rows to read from file at a time
        """
        if kwargs.get('if_exists') != 'append' and kwargs.get('if_exists') is not None:
            self._logger.warning('only append is supported with large dataframes')
            self._logger.info('if user is trying to replace or fail use infer_to_snowflake')
            return
        if kwargs.get('if_exists') is None:
            self._logger.info(f'if_exisits defaults to append to {table_name}')
        if_exists = 'append'
        for chunk in pd.read_csv(f'{file_path}', chunksize=chunk_size):
            self.infer_to_snowflake(chunk,
                                    table_name=table_name.upper(),
                                    if_exists=if_exists)
            del chunk
            gc.collect()

    def pandas_to_snowflake(self,
                            df: 'pd.DataFrame',
                            table_name: str,
                            table_check: str,
                            chunk_size: int = None,
                            create_table: bool = False,
                            create_statement: str = None,
                            parallel: int = 4):
        """
        Function that expect a table has been created as the infer function mis
        allocates the size of columns and isn't best practice not chooses the dytpes
        of a column, but is much easier to call upon ``infer_to_snowflake`` recommended
        when in development.
        Args:
        * df (pd.DataFrame): Dataframe that is going to be sent to snowflake
        * table_name (str): Table name being created inside of snowflake
        * chunk_size (int, optional): Number of elements to be inserted once, if not provided all elements will be dumped once.
                                        Defaults to None.
        * create_table (bool, optional): If true will create a table with create_statement. Defaults to False.
        * create_statement (str, optional): Give create table statement and the table will
                                            be created from this string. Defaults to None.
        * parallel (int, optional): Number of threads to be used when uploading chunks. Defaults to None.
        """
        _connection = connector.connect(user=self.sfUser,
                                        password=self.sfPswd,
                                        account=self.sfAccount,
                                        warehouse=self.sfWarehouse,
                                        database=self.sfDatabase,
                                        schema=self.sfSchema)
        if create_table:
            self._logger.info('creating table')
            self.run_str_query(create_statement)
        insert_areas = ['INSERT_TABLE_NAME_HERE', 'INSERET_DATABASE_HERE', 'INSERT_SCHEMA_HERE']
        inserts = [table_name, self.sfDatabase, self.sfSchema]
        for k, v in zip(insert_areas, inserts):
            table_check = table_check.replace(k, v)
        check = self.run_str_query(table_check)
        if check.shape[0] == 0:
            self._logger.error('table not created')
            raise
        try:
            df.columns = [x.upper() for x in df.columns]
            response, nchunks, nrows, _ = write_pandas(conn=_connection, df=df, table_name=table_name.upper(),
                                                       chunk_size=chunk_size, parallel=parallel)
            self._logger.info(f'upload was a success: {response} & the number of rows loaded {nrows}')
            if nrows != df.shape[1]:
                self._logger.warning(f'Dataframe failed to load {nrows - df.shape[1]}')
        except Exception as e:
            self._logger.error(f'{e}')
            raise
        self._logger.info(f'dataframe uploaded to {self.sfDatabase}.{table_name}')

    def execute_concurrent_query(self, query_path: str) -> list:
        """
        Give a user the ability to call multiple sql commands to
        snowflake on a concurrent method giving each command a thread
        to communicate to snowflake to allow snowflake to do more than
        one command at a time as the current methods wait until each query
        is completed.

        Args:
        * query_path (str): file path

        Returns:
        * List: Returns Snowflake Response in a list
        """
        sql_commands = self.parse_sql_file(query_path, delimiter=';')
        return_df_list = []
        with concurrent.futures.ThreadPoolExecutor(max_workers=len(sql_commands)) as executor:
            future_to_url = {executor.submit(self.single_cmd, self, sql_command): sql_command for sql_command in sql_commands}
            for future in concurrent.futures.as_completed(future_to_url):
                sql_command = future_to_url[future]
                try:
                    data = future.result()
                    if data is None:
                        logging.info('snowflake has no response for this action')
                    else:
                        return_df_list.append(data)
                except Exception as exc:
                    return_df_list.append(exc)
                    logging.info('%r generated an exception: %s' % (sql_command, exc))
        return return_df_list

# How to Use

## Start Snowflake Connection

In [5]:
show_doc(SnowflakeConnect)

<h2 id="SnowflakeConnect" class="doc_header"><code>class</code> <code>SnowflakeConnect</code><a href="" class="source_link" style="float:right">[source]</a></h2>

> <code>SnowflakeConnect</code>(**`sfAccount`**:`str`, **`sfUser`**:`str`, **`sfPswd`**:`str`, **`sfWarehouse`**:`str`, **`sfDatabase`**:`str`, **`sfSchema`**:`str`, **`sfRole`**:`str`, **`delimiter`**:`str`=*`';'`*, **`logger`**=*`None`*)

Class that holds basic snowflake functionality including testing connection
and running queries.

In [6]:
sf = SnowflakeConnect(sfAccount=os.environ['sfAccount'],
                   sfUser=os.environ['sfUser'],
                   sfPswd=os.environ['sfPswd'],
                   sfWarehouse=os.environ['sfWarehouse'],
                   sfDatabase=os.environ['sfDatabase'],
                   sfSchema=os.environ['sfSchema'],
                   sfRole=os.environ['sfRole'])
assert sf.test_connection() == None, 'anything else the connection has failed'

INFO:__main__:sqlalchemy snowflake engine created
INFO:__main__:connection to snowflake successful
INFO:__main__:sqlalchemy snowflake engine created
INFO:__main__:connection to snowflake successful


## Pandas to Snowflake

In [7]:
#hide
from sdsde.utils.parseyaml import *
from sdsde.imports import *
sf.run_str_query('DROP TABLE IF EXISTS sdsdequerytest;')
__file__ = './'
dict1 = [{'ecid': 150, 'home': 'CA', 'avg_visits': 0.20, 'LTR': 6},
         {'ecid': 151, 'home': 'LA', 'avg_visits': 10, 'LTR': 2},
         {'ecid': 160, 'home': 'CO', 'avg_visits': 0.56, 'LTR': 4},
         {'ecid': 100, 'home': 'LA', 'avg_visits': 2.0, 'LTR': 3}]
df = pd.DataFrame(dict1)
yaml = ParseYaml(os.path.join(os.path.dirname(__file__), 'testing/snowflake.yaml')).get_yaml(['test'])

INFO:__main__:testing connection
INFO:__main__:sqlalchemy snowflake engine created
INFO:__main__:executing query
INFO:__main__:data loaded from snowflake
INFO:__main__:connection to snowflake has been turned off
INFO:__main__:Drop statement executed successfully (SDSDEQUERYTEST already dropped).


### pandas_to_snowflake

In [8]:
show_doc(SnowflakeConnect.pandas_to_snowflake)

<h4 id="SnowflakeConnect.pandas_to_snowflake" class="doc_header"><code>SnowflakeConnect.pandas_to_snowflake</code><a href="__main__.py#L346" class="source_link" style="float:right">[source]</a></h4>

> <code>SnowflakeConnect.pandas_to_snowflake</code>(**`df`**:`DataFrame`, **`table_name`**:`str`, **`table_check`**:`str`, **`chunk_size`**:`int`=*`None`*, **`create_table`**:`bool`=*`False`*, **`create_statement`**:`str`=*`None`*, **`parallel`**:`int`=*`4`*)

Function that expect a table has been created as the infer function mis
allocates the size of columns and isn't best practice not chooses the dytpes
of a column, but is much easier to call upon ``infer_to_snowflake`` recommended
when in development.
Args:
* df (pd.DataFrame): Dataframe that is going to be sent to snowflake
* table_name (str): Table name being created inside of snowflake
* chunk_size (int, optional): Number of elements to be inserted once, if not provided all elements will be dumped once.
                                Defaults to None.
* create_table (bool, optional): If true will create a table with create_statement. Defaults to False.
* create_statement (str, optional): Give create table statement and the table will
                                    be created from this string. Defaults to None.
* parallel (int, optional): Number of threads to be used when uploading chunks. Defaults to None.

In [9]:
sf.pandas_to_snowflake(df=df,
                       table_name='sdsdequerytest',
                       table_check=ParseYaml('testing/snowflake.yaml').get_yaml(['test']).get('check_table'),
                       chunk_size = None,
                       create_table= True,
                       create_statement= yaml.get('create_query_table'))
df = sf.run_str_query("SELECT * FROM sdsdequerytest")
assert df.shape == (4, 4), 'Query 4 observations w/ 4 columns from sdsdequerytest'

INFO:__main__:creating table
INFO:__main__:testing connection
INFO:__main__:sqlalchemy snowflake engine created
INFO:__main__:executing query
INFO:__main__:data loaded from snowflake
INFO:__main__:connection to snowflake has been turned off
INFO:__main__:Table SDSDEQUERYTEST successfully created.
INFO:__main__:testing connection
INFO:__main__:sqlalchemy snowflake engine created
INFO:__main__:executing query
INFO:__main__:data loaded from snowflake
INFO:__main__:connection to snowflake has been turned off
INFO:__main__:upload was a success: True & the number of rows loaded 4
INFO:__main__:dataframe uploaded to MACHINELEARNINGOUTPUTS.sdsdequerytest
INFO:__main__:testing connection
INFO:__main__:sqlalchemy snowflake engine created
INFO:__main__:executing query
INFO:__main__:data loaded from snowflake
INFO:__main__:connection to snowflake has been turned off


### infer_to_snowflake

In [10]:
show_doc(SnowflakeConnect.infer_to_snowflake)

<h4 id="SnowflakeConnect.infer_to_snowflake" class="doc_header"><code>SnowflakeConnect.infer_to_snowflake</code><a href="__main__.py#L260" class="source_link" style="float:right">[source]</a></h4>

> <code>SnowflakeConnect.infer_to_snowflake</code>(**`df`**:`DataFrame`, **`table_name`**:`str`, **`if_exists`**:`str`=*`'append'`*, **`dtype`**:`dict`=*`None`*, **`chunk_size`**:`int`=*`None`*)

Args:
* df (pd.DataFrame): The dataframe that is going to snowflake
* table_name (str): Table name that will be created in snowflake
* if_exists (str, optional): How to behave if the table already exists.
                            * fail: Raise a ValueError.
                            * replace: Drop the table before inserting new values.
                            * append: Insert new values to the existing table.
                            Defaults to 'fail'.
* dtype (dict, optional): Specifying the datatype for columns. If a dictionary is used, the
                        keys should be the column names and the values should be the
                        SQLAlchemy types or strings for the sqlite3 legacy mode. If a
                        scalar is provided, it will be applied to all columns.
                        Defaults to None.
* chunk_size (int, optional): Specify the number of rows in each batch to be written
                            at a time. By default, all rows will be written at once.
                            Defaults to None.

In [11]:
sf.run_str_query('DROP TABLE IF EXISTS sdsdequerytest;')
sf.infer_to_snowflake(df,
                      table_name='sdsdequerytest',
                      if_exists='append')
df = sf.run_str_query("SELECT * FROM sdsdequerytest");
assert df.shape == (4, 4), 'Query 4 observations w/ 4 columns from sdsdequerytest'
sf.run_str_query('DROP TABLE IF EXISTS sdsdequerytest;')

INFO:__main__:testing connection
INFO:__main__:sqlalchemy snowflake engine created
INFO:__main__:executing query
INFO:__main__:data loaded from snowflake
INFO:__main__:connection to snowflake has been turned off
INFO:__main__:SDSDEQUERYTEST successfully dropped.
INFO:__main__:Begining upload to sdsdequerytest
INFO:__main__:sqlalchemy snowflake engine created
/home/azureuser/miniconda3/envs/dsde_upgrade/lib/python3.8/site-packages/pandas/io/sql.py:1685: UserWarning: The provided table name 'SDSDEQUERYTEST' is not found exactly as such in the database after writing the table, possibly due to case sensitivity issues. Consider using lower case table names.
  warnings.warn(msg, UserWarning)
INFO:__main__:Dataframe uploaded to sdsdequerytest
INFO:__main__:testing connection
INFO:__main__:sqlalchemy snowflake engine created
INFO:__main__:executing query
INFO:__main__:data loaded from snowflake
INFO:__main__:connection to snowflake has been turned off
INFO:__main__:testing connection
INFO:__ma

### large_table_infer_sf

In [12]:
show_doc(SnowflakeConnect.large_table_infer_sf)

<h4 id="SnowflakeConnect.large_table_infer_sf" class="doc_header"><code>SnowflakeConnect.large_table_infer_sf</code><a href="__main__.py#L290" class="source_link" style="float:right">[source]</a></h4>

> <code>SnowflakeConnect.large_table_infer_sf</code>(**`df`**:`DataFrame`, **`table_name`**:`str`, **`split`**:`int`, **`if_exists`**:`str`=*`'append'`*, **`dtype`**:`dict`=*`None`*, **`chunk_size`**:`int`=*`None`*)

This take infer_to_snowflake and give the ability to split the loads to snowflake
this isn't tested for efficiency, but might be useful when not wanting to wait
for a 100M+ memory object to make it to snowflake. A user can split this into
as many slices as they choose and it will fail if there is something wrong with
the upload.

Args:
* df (pd.DataFrame): in memory pandas dataframe
* table_name (str): table name for table being created
* split (int): how many splits to split the data frame up in

In [13]:
sf.run_str_query('DROP TABLE IF EXISTS sdsdequerytest;')
sf.large_table_infer_sf(df=df, table_name='sdsdequerytest', split=1, if_exists='append')
df = sf.run_str_query("SELECT * FROM sdsdequerytest");
assert df.shape == (4, 4), 'Query 4 observations w/ 4 columns from sdsdequerytest'
sf.run_str_query('DROP TABLE IF EXISTS sdsdequerytest;')

INFO:__main__:testing connection
INFO:__main__:sqlalchemy snowflake engine created
INFO:__main__:executing query
INFO:__main__:data loaded from snowflake
INFO:__main__:connection to snowflake has been turned off
INFO:__main__:Drop statement executed successfully (SDSDEQUERYTEST already dropped).
INFO:__main__:Begining upload to SDSDEQUERYTEST
INFO:__main__:sqlalchemy snowflake engine created
INFO:__main__:Dataframe uploaded to SDSDEQUERYTEST
INFO:__main__:testing connection
INFO:__main__:sqlalchemy snowflake engine created
INFO:__main__:executing query
INFO:__main__:data loaded from snowflake
INFO:__main__:connection to snowflake has been turned off
INFO:__main__:testing connection
INFO:__main__:sqlalchemy snowflake engine created
INFO:__main__:executing query
INFO:__main__:data loaded from snowflake
INFO:__main__:connection to snowflake has been turned off
INFO:__main__:SDSDEQUERYTEST successfully dropped.


## SQL Execution

The point of this function is to allow a user to make create a table/View and then query the table with a specific command. This can be done the same way with ``execute_file_query``, but the final command will be a the end of the file.

In [14]:
show_doc(SnowflakeConnect.execute_file)

<h4 id="SnowflakeConnect.execute_file" class="doc_header"><code>SnowflakeConnect.execute_file</code><a href="__main__.py#L156" class="source_link" style="float:right">[source]</a></h4>

> <code>SnowflakeConnect.execute_file</code>(**`query_path`**:`str`, **`sql`**:`str`=*`None`*, **`role`**=*`None`*, **`schema`**=*`None`*, **`password`**=*`None`*, **`user`**=*`None`*, **`warehouse`**=*`None`*, **`database`**=*`None`*)

will run sql file or txt file that is seperated by self.delimenter
with the option to add a custom sql command if you run an sql command
then the sql command will be the returned response

Args:
* query_path (str): location to sql/txt file to execute. Defaults to None.
* sql (str, optional): Optional Command to add on top of the file being executed. Defaults to None.

Returns:
* pd.DataFrame: returns query results

In [15]:
df = sf.execute_file(query_path='testing/view_test.sql')
assert df.status.values[0] == 'Table SDSDE_TEST successfully created.', 'Snowflake changed or credintials did'

INFO:__main__:query being parsed
INFO:__main__:sqlalchemy snowflake engine created
INFO:__main__:sql or txt file excecuted
INFO:__main__:connection to snowflake has been turned off


In [16]:
df = sf.execute_file(sql='SELECT * FROM MACHINELEARNINGOUTPUTS.DEV.sdsde_test WHERE IsCurrent = 1', query_path='testing/view_test.sql')
assert df.shape == (1, 38), 'Query 10 observations w/ 25 columns from DIMCUSTOMER'
sf.run_str_query('DROP TABLE IF EXISTS sdsde_test;')

INFO:__main__:query being parsed
INFO:__main__:sqlalchemy snowflake engine created
INFO:__main__:sql or txt file excecuted
INFO:__main__:Last query response                                    status
0  Table SDSDE_TEST successfully created.
INFO:__main__:data is loaded from snowflake
INFO:__main__:connection to snowflake has been turned off
INFO:__main__:testing connection
INFO:__main__:sqlalchemy snowflake engine created
INFO:__main__:executing query
INFO:__main__:data loaded from snowflake
INFO:__main__:connection to snowflake has been turned off
INFO:__main__:SDSDE_TEST successfully dropped.


In [17]:
show_doc(SnowflakeConnect.run_str_query)

<h4 id="SnowflakeConnect.run_str_query" class="doc_header"><code>SnowflakeConnect.run_str_query</code><a href="__main__.py#L119" class="source_link" style="float:right">[source]</a></h4>

> <code>SnowflakeConnect.run_str_query</code>(**`query`**:`str`, **`role`**=*`None`*, **`schema`**=*`None`*, **`password`**=*`None`*, **`user`**=*`None`*, **`warehouse`**=*`None`*, **`database`**=*`None`*)

This function will connect to snowflake and run a query that is passed
in as a string.
Args:
* query (str): SQL Query String
Returns:
* pd.DataFrame/ None: dependent on the query string

In [18]:
df = sf.run_str_query("SELECT * FROM MACHINELEARNINGOUTPUTS.DEV.DL_LTR LIMIT 100")
assert df.shape == (100, 8), 'Query 10 observations w/ 38 columns from DIMCUSTOMER'

INFO:__main__:testing connection
INFO:__main__:sqlalchemy snowflake engine created
INFO:__main__:executing query
INFO:__main__:data loaded from snowflake
INFO:__main__:connection to snowflake has been turned off


## Concurrent Query Calls

In [19]:
show_doc(SnowflakeConnect.execute_concurrent_query)

<h4 id="SnowflakeConnect.execute_concurrent_query" class="doc_header"><code>SnowflakeConnect.execute_concurrent_query</code><a href="__main__.py#L398" class="source_link" style="float:right">[source]</a></h4>

> <code>SnowflakeConnect.execute_concurrent_query</code>(**`query_path`**:`str`)

Give a user the ability to call multiple sql commands to
snowflake on a concurrent method giving each command a thread
to communicate to snowflake to allow snowflake to do more than
one command at a time as the current methods wait until each query
is completed.

Args:
* query_path (str): file path

Returns:
* List: Returns Snowflake Response in a list

**Question**:

    Why is this something that you will need in your daily work life?
    
**Answer**

   1. This can be used for reporting where there is a need to create multi tables at a time while all the ancillary tables are already ready to be queried. 
   


**What this saves?**

This gives you the ability to not call ``run_str_query`` multiple times and will do what ``execute_file`` does, but it will do it in concurrency so that snowflake query engine will be able to run multiple jobs with out waiting for each one to complete before finishing. This would save the ML_Pipeline 10-15 minutes off it's run if it was a part of the library during it's creation.

This function will not be an extremely popular function in all reality because the majority of the time we will be query the feature store or complex CTE query logic that doesn't actually need this functionality, but when threading multiple calls is of interest of a user it will be possible with this.

In [20]:
df = sf.execute_concurrent_query('./testing/multi.sql')

INFO:root:new thread started
INFO:root:new thread started
INFO:__main__:testing connection
INFO:root:new thread started
INFO:__main__:testing connection
INFO:__main__:sqlalchemy snowflake engine created
INFO:__main__:testing connection
INFO:__main__:sqlalchemy snowflake engine created
INFO:__main__:sqlalchemy snowflake engine created
INFO:__main__:executing query
INFO:__main__:executing query
INFO:__main__:executing query
INFO:__main__:data loaded from snowflake
INFO:__main__:connection to snowflake has been turned off
INFO:root:Finishing SELECT ecid,	likelihoodtoreturnrate,	likelihoodtoreturnlabel FROM MACHINELEARNINGOUTPUTS.DEV.DL_LTR LIMIT 100;
INFO:__main__:data loaded from snowflake
INFO:__main__:connection to snowflake has been turned off
INFO:__main__:Table SDSDE_TEST_2 successfully created.
INFO:root:Finishing CREATE OR REPLACE TABLE sdsde_test_2 AS SELECT ecid,	likelihoodtoreturnrate,	likelihoodtoreturnlabel FROM MACHINELEARNINGOUTPUTS.DEV.DL_LTR LIMIT 100;

INFO:root:snowflak

This is just going to clean up the tables that were created and just another example of executing more than on SQL command in  an concurrent approach

In [22]:
df = sf.execute_concurrent_query('./testing/clean_multi.sql')

INFO:root:new thread started
INFO:root:new thread started
INFO:__main__:testing connection
INFO:__main__:testing connection
INFO:__main__:sqlalchemy snowflake engine created
INFO:__main__:sqlalchemy snowflake engine created
INFO:__main__:executing query
INFO:__main__:executing query
INFO:__main__:data loaded from snowflake
INFO:__main__:connection to snowflake has been turned off
INFO:__main__:SDSDE_TEST successfully dropped.
INFO:root:Finishing DROP TABLE sdsde_test;

INFO:root:snowflake has no response for this action
INFO:__main__:data loaded from snowflake
INFO:__main__:connection to snowflake has been turned off
INFO:__main__:SDSDE_TEST_2 successfully dropped.
INFO:root:Finishing DROP TABLE sdsde_test_2;
INFO:root:snowflake has no response for this action


# Create

In [23]:
#hide
from nbdev.export import notebook2script
notebook2script()

Converted 01_azure.ipynb.
Converted 02_utils_dataframes.ipynb.
Converted 02_utils_parseyaml.ipynb.
Converted 02_utils_stfp.ipynb.
Converted 02_utils_traininghelpers.ipynb.
Converted 02_utils_traininghelpers_fastai.ipynb.
Converted 03_dstools_preparedata.ipynb.
Converted 04_snowflake_copyinto.ipynb.
Converted 04_snowflake_copyinto2.ipynb.
Converted 04_snowflake_query.ipynb.
Converted 05_azure_wrappers.ipynb.
Converted 06_modeling_inference.ipynb.
Converted 06_modeling_inference_fastai.ipynb.
Converted 06_modeling_premodel.ipynb.
Converted 06_modeling_preprocessing.ipynb.
Converted 06_modeling_preprocessing_fastai.ipynb.
Converted 06_modeling_training.ipynb.
Converted 06_modeling_training_fastai.ipynb.
Converted 07_Binary_Classification_Fastai_Example_Notebook.ipynb.
Converted index.ipynb.
